In [102]:
# Set up the workspace
using SumOfSquares, JuMP, PolyJuMP, DynamicPolynomials, MultivariatePolynomials
using Mosek, CSDP#, SCS
using Plots
gr()

include("../src/NormalSoS.jl")
using NormalSoS

# Linear Example

As a first test case we will examine the linear example of Zhou _et al._ (2012). For a linear system the dynamics is expressed in terms of a matrix $A$, and the properties of the decomposition are determined by some well known properties of this matrix.
\begin{equation}
    \dot{x} = f(x) = Ax.
\end{equation}
If $A$ is a normal matrix, then an orthogonal decomposition should be achieveable via a symmetric-antisymmetric decomposition. In this case the quasi-potential is given by:
\begin{equation}
    V(x) = -\frac{1}{2}((A+A^*)x,x),
\end{equation}
where the brackets denote an inner product.

If $A$ is not normal, then the quasipotential is given by the slightly more complicated expression:
\begin{equation}
    V(x) = -\frac{1}{2}\left( \left( \int_0^\infty \exp(At) \exp(A^*t)\,\text{d}t\right)^{-1}x, x \right).
\end{equation}
Such a quasipotential will not satisfy a normal decomposition. A useful comparison can therefore be made between this analytical expression for the quasipotential and that resulting from the SoS method for non-normal linear systems.

In [103]:
# A 2D normal example - WORKS
@polyvar x1[1:2]
A = [-5.0 0.2;
     0.2 -1.0];
F1(x::Vector) = A*x;
f1 = F1(x1);
U1 = NormalSoS.normdecomp(f1,x1, MosekSolver(),0);
# V1 = -0.25*dot((A+A')*x1,x1);

(λ,u) = eig(A);
(~,v) = eig(A');
P = diagm(λ);
M = inv(u)*v
for i=1:2, j=1:2
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V1 = 0.25*dot(inv(u*M*inv(v))*x1,x1)

@show(U1);    @show(V1);

Chosen basis as:
DynamicPolynomials.Monomial{true}[x11^2, x11x12, x12^2, 1]


LoadError: [91mMethodError: no method matching redirect_stdout(::IJulia.IJuliaStdio{Base.PipeEndpoint})[0m
Closest candidates are:
  redirect_stdout() at stream.jl:1034
  redirect_stdout([91m::Union{Base.LibuvStream, IOStream}[39m) at stream.jl:1028
  redirect_stdout([91m::Function[39m, [91m::Any[39m) at stream.jl:1082[39m

In [85]:
# A 3D normal example - WORKS
@polyvar x2[1:3]
A = [-5.0 0.0 0.2;
     0.0 -1.5 0.0;
     0.2 0.0 -1.0];
F2(x::Vector) = A*x;
f2 = F2(x2);
U2 = NormalSoS.normdecomp(f2,x2, MosekSolver(),0);
# V2 = -0.25*dot((A+A')*x2,x2);

(λ,u) = eig(A);
(~,v) = eig(A');
P = diagm(λ);
M = inv(u)*v
for i=1:3, j=1:3
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V2 = 0.25*dot(inv(u*M*inv(v))*x2,x2)

@show(U2);    @show(V2);

Chosen basis as:
DynamicPolynomials.Monomial{true}[x21^2, x21x23, x22^2, x23^2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 279             
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optim

In [86]:
# A 2D non-normal example - WORKS
@polyvar x3[1:2]
A = [-5.0 3.8;
     0.2 -1.0];
F3(x::Vector) = A*x;
f3 = F3(x3);
U3 = NormalSoS.normdecomp(f3,x3, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
P = diagm(λ);
M = inv(u)*v
for i=1:2, j=1:2
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V3 = 0.25*dot(inv(u*M*inv(v))*x3,x3)

@show(U3);    @show(V3);
NormalSoS.checknorm(f3,U3,x3)

Chosen basis as:
DynamicPolynomials.Monomial{true}[x31^2, x31x32, x32^2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 98              
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization

2.091013131651683e-6

In [101]:
# A 3D non-normal example
@polyvar x4[1:3]
A = [-5.0 0.0 0.2;
     0.0 -1.5 0.0;
     0.2 0.0 -1.0];
F4(x::Vector) = A*x;
f4 = F4(x4);
U4 = NormalSoS.normdecomp(f4,x4, MosekSolver(),0);
# U4 = NormalSoS.normopt2(f4,x4,monomials(V4+1),MosekSolver())

(λ,u) = eig(A);
(~,v) = eig(A');
P = diagm(λ);
M = inv(u)*v
for i=1:3, j=1:3
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V4 = 0.25*dot(inv(u*M*inv(v))*x4,x4)

@show(U4);    @show(V4);
NormalSoS.checknorm(f4,U4,x4)

Chosen basis as:
DynamicPolynomials.Monomial{true}[x41^2, x41x43, x42^2, x43^2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 279             
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optim

6.853419249130022e-7

In [100]:
# @show(monomials(U4));
@show(monomials(V4+1));
@show(monomials(x4,0:2))

monomials(V4 + 1) = DynamicPolynomials.Monomial{true}[x41^2, x41x42, x41x43, x42^2, x42x43, x43^2, 1]
monomials(x4, 0:2) = DynamicPolynomials.Monomial{true}[x41^2, x41x42, x41x43, x42^2, x42x43, x43^2, x41, x42, x43, 1]


10-element DynamicPolynomials.MonomialVector{true}:
 x41^2 
 x41x42
 x41x43
 x42^2 
 x42x43
 x43^2 
 x41   
 x42   
 x43   
 1     

In [72]:
Pkg.update()

INFO: Updating METADATA...
INFO: Computing changes...


LoadError: [91mresolve is unable to satisfy package requirements.
  The problem was detected when trying to find a feasible version
  for package DiffEqPhysics.
  However, this only means that package DiffEqPhysics is involved in an
  unsatisfiable or difficult dependency relation, and the root of
  the problem may be elsewhere.
  (you may try increasing the value of the JULIA_PKGRESOLVE_ACCURACY
   environment variable)
[39m